In [ ]:
from yftools import Data
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook'

def _prepare_data(df):
    try:
        df.columns = df.columns.astype('str')
        rev = df.loc['TotalRevenue'].dropna()
        netinc = df.loc['NetIncome'].dropna()
        fcf = df.loc['FreeCashFlow'].dropna()
        assets = df.loc['TotalAssets'].dropna()
        liab = df.loc['CurrentLiabilities'].dropna()

        debt_assets = liab/assets
        fcf_margin = fcf/rev*100
        net_margin = netinc/rev*100
        rev_change = rev.pct_change().fillna(0.0)*100

        data = {
            'debtAssets': debt_assets,
            'fcfMargin': fcf_margin,
            'netMargin': net_margin,
            'changeInRevenue': rev_change
        }
    except KeyError:
        data = None
    
    return data

def plot(df, ticker=''):
    data = _prepare_data(df)
    if data is None:
        return
    
    # Moderne Farbpalette
    colors = {
        'fcf': '#00D9FF',      # Cyan
        'net': '#FF6B9D',      # Pink
        'revenue_pos': '#00FF88',  # Grün
        'revenue_neg': '#FF4444',  # Rot
        'debt': '#FFB800'      # Orange
    }
    
    fig = make_subplots(
        cols=1, rows=3,
        row_heights=[0.4, 0.3, 0.3],
        subplot_titles=(
            'Margen (%)',
            'Umsatzwachstum (%)',
            'Schulden/Assets Ratio'
        ),
        vertical_spacing=0.12
    )
    
    # Margen (FCF & Net)
    fig.add_trace(
        go.Bar(
            x=data['fcfMargin'].index,
            y=data['fcfMargin'],
            name='FCF Marge',
            marker=dict(
                color=colors['fcf'],
                line=dict(color=colors['fcf'], width=1.5)
            ),
            hovertemplate='<b>%{x}</b><br>FCF Marge: %{y:.2f}%<extra></extra>'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=data['netMargin'].index,
            y=data['netMargin'],
            name='Netto Marge',
            marker=dict(
                color=colors['net'],
                line=dict(color=colors['net'], width=1.5)
            ),
            hovertemplate='<b>%{x}</b><br>Netto Marge: %{y:.2f}%<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Umsatzwachstum
    rev_colors = [colors['revenue_pos'] if i >= 0 else colors['revenue_neg'] 
                  for i in data['changeInRevenue']]
    
    fig.add_trace(
        go.Bar(
            x=data['changeInRevenue'].index,
            y=data['changeInRevenue'],
            name='Umsatzänderung',
            marker=dict(
                color=rev_colors,
                line=dict(color='rgba(255,255,255,0.3)', width=1)
            ),
            hovertemplate='<b>%{x}</b><br>Änderung: %{y:.2f}%<extra></extra>',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # Zero-Linie für Umsatzwachstum
    fig.add_hline(
        y=0, line_dash="dash", 
        line_color="rgba(255,255,255,0.3)", 
        line_width=1,
        row=2, col=1
    )
    
    # Schulden/Assets Ratio
    fig.add_trace(
        go.Scatter(
            x=data['debtAssets'].index,
            y=data['debtAssets'],
            name='Schulden/Assets',
            mode='lines+markers',
            line=dict(color=colors['debt'], width=3),
            marker=dict(
                size=8,
                color=colors['debt'],
                line=dict(color='white', width=2)
            ),
            fill='tozeroy',
            fillcolor=f'rgba(255, 184, 0, 0.1)',
            hovertemplate='<b>%{x}</b><br>Ratio: %{y:.3f}<extra></extra>'
        ),
        row=3, col=1
    )
    
    # Layout-Anpassungen
    title_text = f'Financial Analysis - {ticker}' if ticker else 'Financial Analysis'
    
    fig.update_layout(
        title=dict(
            text=title_text,
            font=dict(size=24, family='Montserrat', color='white'),
            x=0.5,
            xanchor='center'
        ),
        template='plotly_dark',
        font=dict(size=12, family='Montserrat'),
        height=800,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor='rgba(255,255,255,0.2)',
            borderwidth=1
        ),
        hovermode='x unified',
        plot_bgcolor='#0a0a0a',
        paper_bgcolor='#121212',
        margin=dict(t=120, b=60, l=60, r=60)
    )
    
    # Achsen-Styling
    for i in range(1, 4):
        fig.update_xaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(255,255,255,0.1)',
            showline=True,
            linewidth=1,
            linecolor='rgba(255,255,255,0.2)',
            row=i, col=1
        )
        fig.update_yaxes(
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(255,255,255,0.1)',
            showline=True,
            linewidth=1,
            linecolor='rgba(255,255,255,0.2)',
            row=i, col=1
        )
    
    # Y-Achsen Beschriftungen
    fig.update_yaxes(title_text="Prozent (%)", row=1, col=1)
    fig.update_yaxes(title_text="Prozent (%)", row=2, col=1)
    fig.update_yaxes(title_text="Ratio", row=3, col=1)
    
    # Subplot-Titel Styling
    for annotation in fig.layout.annotations:
        annotation.font.size = 14
        annotation.font.color = 'rgba(255,255,255,0.8)'
    
    fig.show()
    
if __name__ == '__main__':
    stock = 'AXON'
    freq = 'quarterly'
    df = Data(stock).fundamentals(freq)
    plot(df, ticker=stock)